In [1]:
import cv2

In [12]:
cap=cv2.VideoCapture(0)

# Setting frame Size
cap.set(3,1366)
cap.set(4,768)

# Load our image template, this is our referece image
image=cv2.imread('Image/test_image.png',0)

while True:
    
    # Get webcam image
    ret,frame=cap.read()
    
    # Get height and width of webcam frame
    w,h = frame.shape[:2]
    
    # Define ROI dimensions
    top_left_x = int(w/1.5)
    top_left_y =int(3*(h/7))
    bottom_right_x=int((w/1.8)*2)
    bottom_right_y = int((h/3)-(h/5.5))
    
    # Draw Rectangle window for our region of interest
    cv2.rectangle(frame,(top_left_x, top_left_y),(bottom_right_x , bottom_right_y),255,3)
    
    # Crop window of operation we defined above
    cropped=frame[bottom_right_y:top_left_y,top_left_x:bottom_right_x]
    
    # Flip frame operation horizontally
    frame=cv2.flip(frame,1)
    
    # Get number of orb matches
    matches =orbDetector(cropped,image)
    
    # Display status string showing the number of correct matches
    cv2.putText(frame,str(matches),(450,600),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),1)
    
    # Our threshold to indicate object detection
    threshold=350
    
    # If matches exceed our threshold then object is detected
    if matches > threshold:
        cv2.rectangle(frame,(top_left_x, top_left_y),(bottom_right_x , bottom_right_y),(0,255,0),3)
        cv2.putText(frame,'object found',(450,150),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),2)
        
        
    cv2.imshow('fa',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [11]:
def orbDetector(img,img_temp):
    # Function that compares input image to template image
    # It then returns number of ORB matches between them
    
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Create ORB detector with 1000 key points and with a scaling pyramid factor of 1.2
    orb=cv2.ORB_create(1000, 1.2)
    
    # Detect key points of original image
    (key1, des1)=orb.detectAndCompute(img, None)
    
    # Detect key points of template image
    (key2, des2)=orb.detectAndCompute(img_temp,None)
    
    # Create matcher
    bf=cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck= True)
    
    # Do matching
    matches=bf.match(des1,des2)
    
    # Sort the matches based on distance
    matches=sorted(matches, key=lambda x:x.distance)
    
    return len(matches)